In [1]:
using PCT

┌ Info: Precompiling PCT [ef708a43-c8a3-43f4-8f65-1f04ee4c5bb0]
└ @ Base loading.jl:1664


In [2]:
f = @pct begin
@space begin
    name=M
    type=(I, I) -> R
    symmetries=((1,2),)
end
@space begin
    name=V
    type=(I,) -> R
end

# (y::V) -> (j::I) -> ((x(i)::V) -> x(i))(y(j))
# (y::V) -> (j::I) -> pullback((x(i)::V) -> x(j) * y(j))
# pullback((x::R) -> x * x + x)
pullback((x::V) -> sum((i), x(i) * x(i)))
end

𝒫((x) -> 
    ∑((i), x(i)*x(i)))

In [3]:
f = inference(f)

𝒫((x) -> 
    ∑((i), x(i)*x(i)))

In [4]:
e_f = evaluate(f)

𝒫((x) -> 
    ∑((i), x(i)*x(i)))

In [5]:
println(verbose(e_f))

Pullback((x::[I->R]->
    ∑((i::I),
        (*
            x::[I->R](i::I)::R,
            x::[I->R](i::I)::R
        )::R
    )::R
::[[I->R]->R])::[[I->R]×R->[I->R]]


In [11]:
p = evaluate_pullback(e_f)

(x, 𝒦) -> 
    (i_1) -> 
        ((x(i_1), 𝒦) -> 
            ∑((i), ((x(i_1), 𝒦) -> 
                ((x(i_1), 𝒦) -> 
                    δ(i_1, i, 𝒦))(x(i_1), x(i)*𝒦)+((x(i_1), 𝒦) -> 
                    δ(i_1, i, 𝒦))(x(i_1), x(i)*𝒦))(x(i_1), 𝒦)))(x(i_1), 𝒦)

In [16]:
evaluate(p)

(x, 𝒦) -> 
    (i_1) -> 
        ∑((i), (δ(i_1, i, x(i)*𝒦)+δ(i_1, i, x(i)*𝒦)))